In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#lecture du dataset 
df=pd.read_csv("dataset_jeux_par_item V2.csv")
df.head(10)

,jeu,Genre,nom_plateforme,nom_fabricant,editeur,developpeur,annee,ttl_sales,dev_country,metacritic_summary,...,igdb_age_rating_category,igdb_age_rating_class,igdb_developer_name,igdb_developer_country,igdb_publisher_name,igdb_publisher_country,igdb_websites_category,igdb_websites_url,twitch_id,twitch_streams_count
0,Wii Sports,Sports,Wii,Nintendo,Nintendo,NINTENDO,2006.0,82.90,Japan,Tennis (1-4 players): Players grab the control...,...,"ESRB, PEGI, CLASS_IND, CERO, USK, GRAC, ACB","ESRB_E, PEGI_7, CLASS_IND_10, CERO_B, USK_12, ...",Nintendo EAD Software Development Group No.2,Japan,Nintendo,Japan,"official, wikipedia, wikia, twitch","http://wiisports.nintendo.com/, https://en.wik...",12232.0,1.0
1,Mario Kart 8 Deluxe,Racing,Nintendo Switch,Nintendo,Nintendo,NINTENDO,2017.0,48.41,Japan,Race and battle your friends in the definitive...,...,"ESRB, PEGI, USK, CLASS_IND, CERO, GRAC, ACB","ESRB_E, PEGI_3, USK_0, CLASS_IND_10, CERO_A, G...",Nintendo EPD,Japan,"Nintendo of America, Nintendo of Europe","United States of America, Germany","wikia, wikipedia, official, twitch, reddit","https://www.mariowiki.com/Mario_Kart_8_Deluxe,...",941530474.0,670.0
2,Super Mario Bros.,Platform,Nintendo (NES),Nintendo,Nintendo,NINTENDO,1985.0,40.24,Japan,NaN,...,"ESRB, PEGI, CLASS_IND, CERO, USK, GRAC, ACB","ESRB_E, PEGI_3, CLASS_IND_10, CERO_A, USK_0, G...","Nintendo, Nintendo EAD, Nintendo R&D4",Japan,Nintendo,Japan,"wikipedia, wikia, twitch, youtube, facebook, t...",https://en.wikipedia.org/wiki/List_of_Game_%26...,509508.0,13.0
3,Animal Crossing: New Horizons,Simulation,Nintendo Switch,Nintendo,Nintendo,NINTENDO,2020.0,40.17,Japan,If the hustle and bustle of modern life’s got ...,...,"ESRB, PEGI, USK, CERO","ESRB_E, PEGI_3, USK_0, CERO_A","Nintendo EPD, Nintendo",Japan,Nintendo,Japan,"official, wikia, wikipedia, twitter, reddit, d...",https://www.nintendo.com/games/detail/animal-c...,509538.0,1655.0
4,Counter-Strike: Global Offensive,Shooter,Microsoft Windows,PC,Valve,VALVE,2012.0,40.00,United States,Counter-Strike: Global Offensive features new ...,...,"ESRB, PEGI, USK, CLASS_IND, ACB","ESRB_M, PEGI_18, USK_18, CLASS_IND_16, ACB_R18","Hidden Path Entertainment, Valve",United States of America,Valve,United States of America,"official, wikia, wikipedia, facebook, youtube,...","http://blog.counter-strike.net/, http://counte...",32399.0,24977.0
5,Mario Kart Wii,Racing,Wii,Nintendo,Nintendo,NINTENDO,2008.0,37.38,Japan,Mario Kart Wii comes with the intuitive Wii Wh...,...,"ESRB, PEGI, CLASS_IND, CERO, ACB","ESRB_E, PEGI_3, CLASS_IND_10, CERO_A, ACB_PG","Nintendo, Nintendo EAD",Japan,Nintendo,Japan,"wikipedia, wikia, official, twitch","https://en.wikipedia.org/wiki/Mario_Kart_Wii, ...",18871.0,193.0
6,PLAYERUNKNOWN'S BATTLEGROUNDS,Shooter,Microsoft Windows,PC,PUBG Corporation,PUBG CORPORATION,2017.0,36.60,Unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Minecraft,Misc,Microsoft Windows,PC,Mojang,MOJANG,2010.0,33.15,Sweden,The game involves players creating and destroy...,...,"ESRB, PEGI, CERO, USK, CLASS_IND, ACB, GRAC","ESRB_E10, PEGI_7, CERO_A, USK_6, CLASS_IND_10,...",Mojang Studios,Sweden,Mojang Studios,Sweden,"official, wikipedia, facebook, twitter, youtub...","https://minecraft.net/, https://en.wikipedia.o...",27471.0,49502.0
8,Wii Sports Resort,Sports,Wii,Nintendo,Nintendo,NINTENDO,2009.0,33.14,Japan,Wii Sports Resort is a collection of fun sport...,...,"ESRB, PEGI, CLASS_IND, CERO, USK, ACB, GRAC","ESRB_E, PEGI_7, CLASS_IND_10, CERO_B, USK_12, ...",Nintendo EAD Software Development Group No.2,Japan,Nintendo,Japan,"official, wikipedia, wikia, twitch","http://www.wiisportsresort.com/, https://en.wi...",19259.0,7.0
9,Pokemon Red / Green / Blue Version,Role-Playing,Game Boy,Nintendo,Nintendo,GAME FREAK,1998.0,31.38,Japan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#colonne année mise en format datetime et remplacement des valeurs manquantes par l'année 1900 suffisamment ancienne pour ne pas affecter les résultats d'analyse
from datetime import datetime as dt 
df['annee']=pd.to_datetime(df['annee'], errors="coerce").fillna(pd.Timestamp("1900"))
df['annee'] = df['annee'].dt.year
df["annee"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 20465 entries, 0 to 20464
Series name: annee
Non-Null Count  Dtype
--------------  -----
20465 non-null  int64
dtypes: int64(1)
memory usage: 160.0 KB


In [ ]:
#Prise en compte des 10K premières lignes 
df_top10K=df.head(10000)

In [ ]:
#selection des éditeurs ayant 3 ou plus de jeux vidéos 
counts = df_top10K['editeur'].value_counts()
selected_publishers = counts[counts >= 3].index
df_top10K= df_top10K[df_top10K['editeur'].isin(selected_publishers)]

In [ ]:
df_top10K.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9591 entries, 0 to 9999
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   jeu                            9591 non-null   object 
 1   Genre                          9591 non-null   object 
 2   nom_plateforme                 9591 non-null   object 
 3   nom_fabricant                  9591 non-null   object 
 4   editeur                        9591 non-null   object 
 5   developpeur                    9591 non-null   object 
 6   annee                          9591 non-null   int64  
 7   ttl_sales                      9591 non-null   float64
 8   dev_country                    9589 non-null   object 
 9   metacritic_summary             5820 non-null   object 
 10  metacritic_user_score          5820 non-null   float64
 11  metacritic_score               5820 non-null   float64
 12  jvc_summmary                   2866 non-null   o

In [ ]:
#slip de la colonne age rating class pour ne garder que les ESRB
df_top10K['ESRB'] = df_top10K['igdb_age_rating_class'].str.split(',').str[0].apply(lambda x: x.strip() if str(x).startswith('ESRB') else pd.NaT)

In [ ]:
df_top10K.columns

Index(['jeu', 'Genre', 'nom_plateforme', 'nom_fabricant', 'editeur',
       'developpeur', 'annee', 'ttl_sales', 'dev_country',
       'metacritic_summary', 'metacritic_user_score', 'metacritic_score',
       'jvc_summmary', 'jvc_user_score', 'jvc_url', 'mixed_user_score',
       'igdb_id', 'igdb_category', 'igdb_follows', 'igdb_hypes',
       'igdb_franchise', 'igdb_platforms', 'igdb_genres', 'igdb_game_modes',
       'igdb_rating', 'igdb_summary', 'igdb_themes', 'igdb_total_rating',
       'igdb_url', 'igdb_age_content_descriptions', 'igdb_age_rating_category',
       'igdb_age_rating_class', 'igdb_developer_name',
       'igdb_developer_country', 'igdb_publisher_name',
       'igdb_publisher_country', 'igdb_websites_category', 'igdb_websites_url',
       'twitch_id', 'twitch_streams_count', 'ESRB'],
      dtype='object')

In [ ]:
#On allège le dataset 
colonnes_a_supprimer=['metacritic_summary','metacritic_user_score', 'metacritic_score',
       'jvc_summmary','igdb_id', 'igdb_category','igdb_franchise', 'igdb_platforms', 'igdb_genres', 'igdb_game_modes', 'igdb_summary', 'igdb_themes','igdb_url', 'igdb_age_content_descriptions', 'igdb_age_rating_category',
       'igdb_age_rating_class', 'igdb_developer_name',
       'igdb_developer_country', 'igdb_publisher_name',
       'igdb_publisher_country', 'igdb_websites_category', 'igdb_websites_url',
       'twitch_id', 'jvc_url','jeu','jvc_user_score', 'jvc_url', 'mixed_user_score','igdb_follows', 'igdb_hypes',
       'igdb_franchise','igdb_rating','igdb_total_rating','twitch_streams_count']
df_top10K = df_top10K.drop(columns=colonnes_a_supprimer)

In [ ]:
df_top10K.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9591 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Genre           9591 non-null   object 
 1   nom_plateforme  9591 non-null   object 
 2   nom_fabricant   9591 non-null   object 
 3   editeur         9591 non-null   object 
 4   developpeur     9591 non-null   object 
 5   annee           9591 non-null   int64  
 6   ttl_sales       9591 non-null   float64
 7   dev_country     9589 non-null   object 
 8   ESRB            5653 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 749.3+ KB


In [ ]:
# encoder les variables catégorielles
cat_vars = ["Genre", "nom_plateforme", "nom_fabricant", "editeur", "developpeur", "dev_country", "ESRB"]
df_top10K_encoded = pd.get_dummies(df_top10K, columns=cat_vars)

In [ ]:
#valeurs manquantes ?
df_top10K_encoded.isna().sum().sum()

0

In [ ]:
#verif 
df_top10K_encoded

,annee,ttl_sales,Genre_Action,Genre_Action-Adventure,Genre_Adventure,Genre_Board Game,Genre_Education,Genre_Fighting,Genre_MMO,Genre_Misc,...,"dev_country_United States, France","dev_country_United States, Japan","dev_country_United States, United Kingdom",dev_country_Unknown,ESRB_ESRB_E,ESRB_ESRB_E10,ESRB_ESRB_EC,ESRB_ESRB_M,ESRB_ESRB_RP,ESRB_ESRB_T
0,1970,82.90,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1970,48.41,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1970,40.24,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1970,40.17,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1970,40.00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1970,0.17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,1970,0.17,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
9997,1970,0.17,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9998,1970,0.17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_error

In [ ]:
#séparation du jeu de données 
X_train, X_test, y_train, y_test = train_test_split(df_top10K_encoded.drop("ttl_sales", axis=1), df_top10K_encoded["ttl_sales"], test_size=0.2, random_state=42)


In [ ]:
X_train, X_test, y_train, y_test

(      annee  Genre_Action  Genre_Action-Adventure  Genre_Adventure  \
 8081   1970             0                       0                0   
 5460   1970             0                       0                0   
 9190   1970             0                       0                0   
 5365   1970             0                       0                0   
 1061   1970             0                       0                0   
 ...     ...           ...                     ...              ...   
 5923   1970             0                       0                0   
 5364   1970             1                       0                0   
 5570   1970             0                       0                0   
 896    1970             1                       0                0   
 7531   1970             0                       0                0   
 
       Genre_Board Game  Genre_Education  Genre_Fighting  Genre_MMO  \
 8081                 0                0               0          0   
 546

In [ ]:
X_train.index

Int64Index([8081, 5460, 9190, 5365, 1061, 9787, 7563, 8879, 1661, 3520,
            ...
            8650, 5764, 4571,  487, 6484, 5923, 5364, 5570,  896, 7531],
           dtype='int64', length=7672)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score

In [ ]:
#utilisation d'un Bagging 
# classificateur Bagging avec un arbre de décision
bc = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=10)

# entraîner le classificateur avec les données d'entraînement
bc.fit(X_train, y_train)

# prédire les étiquettes de classe pour les données de test
y_pred = bc.predict(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
#Affichage de la matrice de confusion 
pd.crosstab(y_test, y_pred, rownames=['classes réelles'],colnames=['classes prédites'])

classes prédites,0.180000,0.182000,0.186000,0.189000,0.190000,0.193000,0.195000,0.197000,0.198000,0.200000,...,11.458600,11.864000,12.839000,14.069000,14.231576,14.985250,15.391000,17.131667,18.447000,27.556000
classes réelles,,,,,,,,,,,,,,,,,,,,,
0.17,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.18,1,1,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
0.19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.20,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0.21,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29.53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30.32,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31.38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
